In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By # css, tag 선택자
from selenium.webdriver.common.keys import Keys # 선택자가 선택을 한 부분에 대해 컨트롤러 (엔터, 탭, ESC, 스페이스바 )
from selenium.webdriver.support.ui import WebDriverWait # 조건이 만족할 때 까지 지능적으로 '대기'
'''
time sleep 을 거는 경우 (복습)
1.웹 마다 늦게 뜨는 경우
2. 봇이 아닌척
'''

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
# '대기' 조건 설정 가능
# HTML 페이지 안에 , 슬라이더가 보일 때, 잠깐 기다려야 점진적으로 보이는 경우, 팝업이 접속 이후 보이는 경우

import pandas as pd
from datetime import datetime
import sys

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

URL = f"https://docs.langchain.com/oss/python/langchain/overview"

options = webdriver.ChromeOptions()
#options.add_argument("--headless") # 로컬 기준 헤드리스를 안주고 실행하는 경우 : 개발할때
# 로컬에서 헤드리스를 주는 경우 (백그라운드) : 운영 할 때  -> 빠르고 GUI 자원 안먹고, 디버깅은 어려움
options.add_argument('--disable-dev-shm-usage')
# 공유 메모리 사용 비활성화 도커같은 가상환경 상에서 안정성 향상, 메모리 부족으로 크래쉬 방지
options.add_argument("--no-sandbox")
# 샌드박스 브라우저 프로세스를 격리 , 시스템 보안을 강화
# 노 샌드 박스 -> 보안 크롤링을 더 자유롭게 할 수 있게함 -> 양날의칼

#Selenium을 사용 
driver = webdriver.Chrome(
    options=options
)

#Selenium을 사용 브라우저 로드
driver.get(URL)
title = driver.title
#driver.close()

print(title)
from bs4 import BeautifulSoup
import json

LangChain overview - Docs by LangChain


In [ ]:
#셀레니움에서 실행한 브라우저를 종료하기
#driver.close()

In [ ]:
#스크레이핑 함수 정의
def get_scraping_soup3(side_link, head_menu_name, side_menu_name):

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    content_area = soup.find("div", id="content-area")

    # 결과를 저장할 리스트와 문단을 임시 저장할 버퍼를 초기화합니다.
    results = []
    current_title = None
    paragraph_buffer = []

    def flush_paragraph_buffer():
        """버퍼에 쌓인 문단들을 하나의 텍스트로 합쳐 results에 추가하는 함수"""
        nonlocal results, current_title, paragraph_buffer
        if paragraph_buffer:
            # 버퍼의 모든 문단을 줄바꿈 두 번으로 연결하여 하나의 텍스트로 만듭니다.
            combined_text = "\n\n".join(paragraph_buffer)
            
            # 합쳐진 텍스트의 길이가 조건에 맞는지 확인합니다.
            if 50 < len(combined_text) < 2000: # 최대 길이를 2000으로 늘렸습니다.
                results.append({
                    "title": current_title,
                    "type": "text",
                    "content": combined_text,
                    "side_link": side_link,
                    "head_menu_name": head_menu_name,
                    "side_menu_name": side_menu_name
                })
            # 버퍼를 비웁니다.
            paragraph_buffer = []

    # content_area가 없을 경우를 대비한 예외 처리
    if not content_area:
        print("id가 'content-area'인 div를 찾을 수 없어 스크레이핑을 건너뜁니다.")
        return []

    # content_area 내에서 제목, 코드, 문단 역할을 하는 span 태그를 찾습니다.
    for tag in content_area.find_all(["h1", "h2", "code", "span"]):
        
        # 1. 태그가 h1 또는 h2인 경우
        #if tag.name in ["h1", "h2"]:
        #if tag.name in ["h1", "h2"] or (tag.name == "span" and tag.has_attr('class') == 'cursor-pointer'):    
        if tag.name in ["h1", "h2"] or (tag.name == "span" and 'cursor-pointer' in tag.get('class', [])):
            # 새로운 제목이 나왔으므로, 이전에 쌓아둔 문단 버퍼를 처리합니다.
            flush_paragraph_buffer()
            # 현재 제목을 업데이트합니다.
            current_title = tag.get_text(strip=True)

        # 2. 태그가 code인 경우 - 이 부분이 수정되었습니다.
        elif tag.name == "code":
            # 코드가 나왔으므로, 이전에 쌓아둔 문단 버퍼를 처리합니다.
            flush_paragraph_buffer()
            
            # 코드 내용을 가져옵니다.
            code_content = tag.get_text(strip=False)
            
            # 코드 내용의 길이가 조건에 맞는지 확인합니다.
            if 20 < len(code_content) < 2000:
                # 코드 내용을 results에 직접 추가합니다.
                results.append({
                    "title": current_title,
                    "type": "code",
                    "content": code_content,
                    "side_link": side_link,
                    "head_menu_name": head_menu_name,
                    "side_menu_name": side_menu_name
                })

        # 3. 태그가 문단(span)인 경우
        elif tag.name == "span" and tag.get('data-as') == 'p':
            # 문단 내용을 버퍼에 추가합니다.
            text = tag.get_text(separator=" ", strip=True)
            if text:
                paragraph_buffer.append(text)

    # 루프가 끝난 후, 버퍼에 아직 처리되지 않은 문단이 남아있을 수 있으므로 마지막으로 처리합니다.
    flush_paragraph_buffer()

    print(f"총 {len(results)}개의 데이터 조각이 수집되었습니다.")

    return results

In [3]:
# 메뉴 링크의 href 속성만 미리 추출
menu_elements = driver.find_elements(By.CSS_SELECTOR, "a.link.nav-tabs-item.group")
menu_hrefs = [elem.get_attribute("href") for elem in menu_elements]

In [4]:
# 사이드바 메뉴 a 태그 클래스에 해당하는 요소들 찾기
#group flex items-center pr-3
sidebar_links = driver.find_elements(By.CSS_SELECTOR, "a.group.flex.items-center.pr-3")

# href 속성만 추출하여 리스트로 저장
sidebar_hrefs = [link.get_attribute("href") for link in sidebar_links]

In [ ]:
len(menu_hrefs)

In [5]:
if __name__ == '__main__':
    results_all = []
    
    # 메뉴 링크와 텍스트를 함께 추출합니다.
    menu_elements = driver.find_elements(By.CSS_SELECTOR, "a.link.nav-tabs-item.group")
    menu_info = [{"href": elem.get_attribute("href"), "text": elem.text} for elem in menu_elements]

    for head_info in menu_info:
        head_href = head_info["href"]
        head_menu_name = head_info["text"] # 헤더 메뉴 이름
        
        driver.get(head_href)
        print(f"상단 메뉴: '{head_menu_name}' ({head_href})")
        
        if "reference" not in head_href:
            print(f"사이드 메뉴 수집 시작 #######")
            
            # 사이드바 링크와 텍스트를 함께 추출합니다.
            sidebar_links = driver.find_elements(By.CSS_SELECTOR, "a.group.flex.items-center.pr-3")
            sidebar_info = []
            for link in sidebar_links:
                try:
                    inner_div = link.find_element(By.CSS_SELECTOR, "div > div")
                    text = inner_div.text
                except NoSuchElementException:
                    text = link.text
                sidebar_info.append({"href": link.get_attribute("href"), "text": text})

            for side_info in sidebar_info:
                side_href = side_info["href"] # 사이드 메뉴 링크
                side_menu_name = side_info["text"] # 사이드 메뉴 이름

                # 예외 처리 로직
                non_valides = [
                    "https://docs.langchain.com/oss/python/langgraph/graph-api",
                    "https://docs.langchain.com/oss/python/langgraph/functional-api",
                ]
                if head_href == "https://docs.langchain.com/oss/python/learn":
                    if side_href in non_valides or "academy.langchain.com" in side_href:
                        print(f"건너뜀: {side_menu_name}")
                        continue
                
                # 페이지 이동 및 스크레이핑
                driver.get(side_href)
                print(f"  -> 사이드바 메뉴: '{side_menu_name}' 스크레이핑 시작...")
                
                # 메타데이터를 인자로 전달합니다.
                scraped_data = get_scraping_soup3(
                    side_link=side_href, 
                    head_menu_name=head_menu_name, 
                    side_menu_name=side_menu_name
                )
                results_all.extend(scraped_data)
                
                print(f"  -> 사이드바 메뉴: '{side_menu_name}' 스크레이핑 끝")
                time.sleep(1)  # 페이지 로딩 대기 시간을 1초로 줄였습니다.

            print(f"'{head_menu_name}' 메뉴의 사이드바 수집 완료 #######")
            time.sleep(1)

상단 메뉴: 'LangChain' (https://docs.langchain.com/oss/python/langchain/overview)
사이드 메뉴 수집 시작 #######
  -> 사이드바 메뉴: 'Overview' 스크레이핑 시작...
총 7개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Overview' 스크레이핑 끝
  -> 사이드바 메뉴: 'Release notes' 스크레이핑 시작...
총 44개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Release notes' 스크레이핑 끝
  -> 사이드바 메뉴: 'Migration guide' 스크레이핑 시작...
총 86개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Migration guide' 스크레이핑 끝
  -> 사이드바 메뉴: 'Install' 스크레이핑 시작...
총 4개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Install' 스크레이핑 끝
  -> 사이드바 메뉴: 'Quickstart' 스크레이핑 시작...
총 18개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Quickstart' 스크레이핑 끝
  -> 사이드바 메뉴: 'Philosophy' 스크레이핑 시작...
총 5개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Philosophy' 스크레이핑 끝
  -> 사이드바 메뉴: 'Agents' 스크레이핑 시작...
총 59개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Agents' 스크레이핑 끝
  -> 사이드바 메뉴: 'Models' 스크레이핑 시작...
총 113개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Models' 스크레이핑 끝
  -> 사이드바 메뉴: 'Messages' 스크레이핑 시작...
총 76개의 데이터 조각이 수집되었습니다.
  -> 사이드바 메뉴: 'Messages' 스크레이핑 끝
  -> 사이드바 메뉴: 'Tools' 스크레이핑 시작...
총

In [7]:
results_all[0]

{'title': 'LangChain overview',
 'type': 'text',
 'content': 'LangChain v1.0 is now available!\n\nFor a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide .\n\nIf you encounter any issues or have feedback, please open an issue so we can improve. To view v0.x documentation, go to the archived content .\n\nLangChain is the easiest way to start building agents and applications powered by LLMs. With under 10 lines of code, you can connect to OpenAI, Anthropic, Google, and more . LangChain provides a pre-built agent architecture and model integrations to help you get started quickly and seamlessly incorporate LLMs into your agents and applications.\n\nWe recommend you use LangChain if you want to quickly build agents and autonomous applications. Use LangGraph , our low-level agent orchestration framework and runtime, when you have more advanced needs that require a combination of deterministic and agentic workflows, heavy customi

In [8]:
len(results_all)

2439

In [ ]:
#results_all 중복 제거 
unique_data = [dict(t) for t in {tuple(d.items()) for d in results_all}]
len(unique_data)

2366

In [10]:
unique_data[0:5]

[{'title': 'File structure',
  'type': 'text',
  'content': 'Below are examples of directory structures for applications:',
  'side_link': 'https://docs.langchain.com/oss/python/langgraph/application-structure',
  'head_menu_name': 'LangGraph',
  'side_menu_name': 'Application structure'},
 {'title': 'Custom stores',
  'type': 'text',
  'content': 'You can also implement your own custom store by extending the BaseStore class. See the store interface documentation for more details.',
  'side_link': 'https://docs.langchain.com/oss/python/integrations/stores',
  'head_menu_name': 'Integrations',
  'side_menu_name': 'Key-value stores'},
 {'title': 'Log to a project',
  'type': 'code',
  'content': 'export LANGSMITH_PROJECT=my-agent-project\n',
  'side_link': 'https://docs.langchain.com/oss/python/langchain/observability',
  'head_menu_name': 'LangChain',
  'side_menu_name': 'Observability'},
 {'title': 'langchain-classic',
  'type': 'code',
  'content': 'uv pip install langchain-classic\n'

In [ ]:
# 코드 블록만 출력
count = 0
for i in unique_data:
    if i['type'] == 'code':
        print(f"type===> {i['type']}")
        print(f"title===> {i['title']}")
        print(f"content===> {i['content']}")
        print(f"side_link===> {i['side_link']}")
        print(f"head_menu_name===> {i['head_menu_name']}")
        print(f"side_menu_name===> {i['side_menu_name']}")
        count += 1
        print("----------------------------------")
#print(f"총 코드 블록 개수: {count}")
print(f" code 의 갯수: {count}")
    

type===> code
title===> Log to a project
content===> export LANGSMITH_PROJECT=my-agent-project

side_link===> https://docs.langchain.com/oss/python/langchain/observability
head_menu_name===> LangChain
side_menu_name===> Observability
----------------------------------
type===> code
title===> langchain-classic
content===> uv pip install langchain-classic

side_link===> https://docs.langchain.com/oss/python/migrate/langchain-v1
head_menu_name===> LangChain
side_menu_name===> Migration guide
----------------------------------
type===> code
title===> ScaNN (Local Index)
content===> pip install scann langchain-community # Requires langchain-community

side_link===> https://docs.langchain.com/oss/python/integrations/providers/google
head_menu_name===> Integrations
side_menu_name===> Google
----------------------------------
type===> code
title===> Prebuilt middleware
content===> HumanInTheLoopMiddleware
side_link===> https://docs.langchain.com/oss/python/releases/langchain-v1
head_menu_name=

In [ ]:
# text 블록만 출력
count = 0
for i in unique_data:
    if i['type'] == 'text':
        print(f"type===> {i['type']}")
        print(f"title===> {i['title']}")
        print(f"content===> {i['content']}")
        print(f"side_link===> {i['side_link']}")
        print(f"head_menu_name===> {i['head_menu_name']}")
        print(f"side_menu_name===> {i['side_menu_name']}")
        count += 1
        print("----------------------------------")
#print(f"총 코드 블록 개수: {count}")
print(f" text 의 갯수: {count}")
    

type===> text
title===> File structure
content===> Below are examples of directory structures for applications:
side_link===> https://docs.langchain.com/oss/python/langgraph/application-structure
head_menu_name===> LangGraph
side_menu_name===> Application structure
----------------------------------
type===> text
title===> Custom stores
content===> You can also implement your own custom store by extending the BaseStore class. See the store interface documentation for more details.
side_link===> https://docs.langchain.com/oss/python/integrations/stores
head_menu_name===> Integrations
side_menu_name===> Key-value stores
----------------------------------
type===> text
title===> Memory Store
content===> A state schema specifies a set of keys that are populated as a graph is executed. As discussed above, state can be written by a checkpointer to a thread at each graph step, enabling state persistence.

But, what if we want to retain some information across threads ? Consider the case of a 

In [ ]:
#데이터 json 형태로 저장
with open('unique_meta.json', 'w', encoding= 'utf-8') as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=4)

In [ ]:
#데이터 불러오기
with open('unique_meta.json', 'r', encoding='utf-8') as f:
    # json.load() 함수를 사용하여 파일의 내용을 파이썬 객체로 불러옵니다.
    loaded_data = json.load(f)

# 데이터가 잘 로드되었는지 확인합니다.
print(f"총 {len(loaded_data)}개의 데이터가 로드되었습니다.")
print("\n--- 로드된 데이터의 첫 번째 항목 ---")
print(loaded_data[0])

총 2366개의 데이터가 로드되었습니다.

--- 로드된 데이터의 첫 번째 항목 ---
{'title': 'File structure', 'type': 'text', 'content': 'Below are examples of directory structures for applications:', 'side_link': 'https://docs.langchain.com/oss/python/langgraph/application-structure', 'head_menu_name': 'LangGraph', 'side_menu_name': 'Application structure'}
